In [ ]:
!pip install nlpaug

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


**Importing Necessary Libraries**

In [ ]:
import pandas as pd
import re
import nlpaug.augmenter.word as naw
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report

**Loading the Dataset**

In [ ]:
#reading the dataset in csv format

data = pd.read_csv('/content/drive/MyDrive/Datasets/gittercom_annotated_data.csv')

**Text Preprocessing**


In [ ]:
def text_preprocessing(text):
    #removing punctuation from text data
    text = re.sub(r'[^\w\s]', '', text)

    #removing special symbols from text data
    text = re.sub(r'[^A-Za-z0-9\s]', '', text)

    #replacing usernames in text data with '<username>'
    text = re.sub(r'@[^\s]+', '<username>', text)

    #replacing url in text data with '<url>' 
    text = re.sub(r'http\S+|www\S+|\S+\.com\S+', '<url>', text)
    
    #replacing code snippets in text data with '<code>' 
    text = re.sub(r'`[^`]+`', '<code>', text)
    
    return text

**Applying Text Preprocessing to the Text Data**

In [ ]:
#applying preprocessing techniques to the text data ('message' column holds all textual communications,i.e., text data)

data['message'] = data['message'].apply(text_preprocessing)

**Counting the Total Number of Instances of Our Dataset**

In [ ]:
data.shape[0]

1000

In [ ]:
#exploring first five rows of our dataset

data.head()

,Channel,message,category
0,Cucumber,Hi Team I just recently upgraded our cucumberj...,fear
1,Cucumber,So github is trying to replace irc P,fear
2,Cucumber,aslakhellesoy Thanks seems like I was using o...,joy
3,Cucumber,Sidkiyassine just call the methods directly,surprise
4,Cucumber,Hello guys need a helpI want to call the run m...,fear


In [ ]:
#exploring the instances of message column that holds text data

data['message']

0      Hi Team I just recently upgraded our cucumberj...
1                   So github is trying to replace irc P
2      aslakhellesoy Thanks  seems like I was using o...
3            Sidkiyassine just call the methods directly
4      Hello guys need a helpI want to call the run m...
                             ...                        
995    Besides the unzip issue which I saw you solved...
996    Hopefully now that the unzip issue is fixed we...
997     I can see that being a problem with my community
998                       which would slow down installs
999         Thanks yet again Ill try your suggestion now
Name: message, Length: 1000, dtype: object

**Splitting the Dataset for Classification**

In [ ]:
X = data['message']
y = data['category']

#splitting the data into training and testing sets (70% for training and 30% for testing)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


**Data Augmentation of training data using Synonym**

In [ ]:
augmented_data = []
augmented_category = []
no_of_targeted_augmentation = 30000

#initializing SynonymAug class of nlp aug library that leverage semantic meaning to substitute word using synonym
data_augmentation = naw.SynonymAug(aug_src='wordnet')

while len(augmented_data) < no_of_targeted_augmentation:
    for message, category in zip(X_train, y_train):
        augmented_text = data_augmentation.augment(message)
        augmented_data.append(augmented_text)
        augmented_category.append(category)
        if len(augmented_data) == no_of_targeted_augmentation:
            break

In [ ]:
X_train = X_train.tolist() + augmented_data[:no_of_targeted_augmentation]
y_train = y_train.tolist() + augmented_category[:no_of_targeted_augmentation]

**Counting the Total Number of Instances of training data After Data Augmentation**

In [ ]:
len(X_train)

30700

**Dropping Any Possible Duplicate Instance(s) of Text Data Due to Performing Data Augmentation**

In [ ]:
data.drop_duplicates(subset='message', inplace=True)

In [ ]:
len(X_train)

30700

**Exploring Augmented Text Data**

In [ ]:
for message, category in zip(X_train, y_train):
    print("message:", message)
    print("category:", category)

Streaming output truncated to the last 5000 lines.
message: ['as well jdubois cannot find the tips folder <url >']
category: sadness
message: ['hi iodin am unravel into cucumberspring124 five springboot130 issues <url> javalangArrayStoreException sunreflectannotationTypeNotPresentExceptionProxy']
category: fear
message: ['ane wish iodin could make this kind of study for free as comfortably but unfortunately its not possible mightily now but it would make up a dreaming come true to put to work full time on a project like this']
category: sadness
message: ['So 1 would just extend that a litle turn maybe with a few notes to that effect and everything would be all good']
category: joy
message: ['Steer and prank look awesome']
category: joy
message: ['Ping pink ping Iu2019d like to incur the jspm work flow for infamous published tonight D Just 380 needs a fix or is it my faulting']
category: joy
message: ['Intelligence community an finis this with this']
category: joy
message: ['good matter

**Converting Augmented Data Back into Strings for Feature Extraction**

In [ ]:
X_train = [' '.join(x) for x in X_train]

**Feature Extraction using TF-IDF**

In [ ]:
vectorizer = TfidfVectorizer()
                         
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

**Training the Dataset using Naive Bayes Classifier**

In [ ]:
#training the dataset using Naive Bayes
model = MultinomialNB()
model.fit(X_train, y_train)

MultinomialNB()

**Testing and Evaluating the Model**

In [ ]:
#predicting on the testing set
y_pred = model.predict(X_test)

#Generating Classification Report of the Model for Computing Precision, Recall and F1-Score for Every Six Emotion Categories
clsf_rprt = classification_report(y_test, y_pred)
print("Classification_report:\n", clsf_rprt)

Classification_report:
               precision    recall  f1-score   support

       anger       0.60      0.11      0.18        28
        fear       0.46      0.67      0.54        78
         joy       0.55      0.58      0.56        88
        love       0.00      0.00      0.00         9
     sadness       0.47      0.47      0.47        88
    surprise       0.00      0.00      0.00         9

    accuracy                           0.49       300
   macro avg       0.35      0.30      0.29       300
weighted avg       0.47      0.49      0.46       300



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#calculating precision score
precision = precision_score(y_test, y_pred, average='weighted')
print("Precision (weighted):", precision)

#calculating recall score
recall = recall_score(y_test, y_pred, average='weighted')
print("Recall (weighted):", recall)

#calculating f1_score score
f1_score = f1_score(y_test, y_pred, average='weighted')
print("f1-score (weighted):", f1_score)

#calculating accuracy score
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Precision (weighted): 0.47474378064559875
Recall (weighted): 0.49
f1-score (weighted): 0.46129186404890316
Accuracy: 0.49


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**Testing the Efficiency of the Model with New Textual Messages Containing Emotion**

In [ ]:
#predicting on new text data containing emotion
text_with_emotion_1 = ["Help Needed! I am getting the following error!!"]
text_with_emotion_1 = vectorizer.transform(text_with_emotion_1)
predicted_labels = model.predict(text_with_emotion_1)
print(predicted_labels)

['fear']


In [ ]:
#predicting on new text data containing emotion
text_with_emotion_2 = ["I was finally able to solve the bug"]
text_with_emotion_2 = vectorizer.transform(text_with_emotion_2)
predicted_labels = model.predict(text_with_emotion_2)
print(predicted_labels)

['joy']
